## Evaluate segmentation algorithm on HRF dataset

In [29]:
import retina_segment
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import mean 
from sklearn.metrics import (classification_report,
                            roc_auc_score,
                            f1_score,
                            matthews_corrcoef,
                            confusion_matrix, 
                            jaccard_score,
                            recall_score,
                            accuracy_score,
                            confusion_matrix)

In [22]:
ORIGINAL_IMG_FOLDER = "../Images/HRF/original/"
SEGMENT_IMG_FOLDER = "../Images/HRF/manual_segment/"

In [23]:
ORIGINAL_IMGs = glob.glob(ORIGINAL_IMG_FOLDER+"*")
SEGMENT_IMGs = glob.glob(SEGMENT_IMG_FOLDER+"*")

In [24]:
def extract_img_ID(path):
    ID = path.split("/")[-1].split(".")[0]
    return ID

In [26]:
y_true_agg = []
y_pred_agg = []

accuracy_list = []
sensitivity_list = []
specificity_list = []

for image_path in ORIGINAL_IMGs:
    ID = extract_img_ID(image_path)
    #print(ID)
    
    # Load original image
    img = cv2.imread(image_path)
    
    # Load segmented image, resize to match with segmentation output, select only green channel and flatten 
    img_segment = cv2.imread(SEGMENT_IMG_FOLDER+ID+".tif")
    img_segment_1000 = cv2.resize(img_segment, (1000, 1000))
    y_true = img_segment_1000[:,:,1].flatten() // 255

    # Make prediction
    y_pred = retina_segment.segmentation(img, resize=True)
    y_pred = y_pred.flatten() // 255
    #print(y_pred.size, y_true.size)

    # Append the flattened arrays to the aggregated lists
    y_true_agg.extend(y_true)
    y_pred_agg.extend(y_pred)

    # Compute metrics for single images
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0

    # Append results
    accuracy_list.append(accuracy)
    sensitivity_list.append(sensitivity)
    specificity_list.append(specificity)

In [32]:
# Average metrics
print("Accuracy:",mean(accuracy_list),
      "Sensitivity:", mean(sensitivity_list),
      "Specificity:",mean(specificity_list))

Accuracy: 0.9524100888888889 Sensitivity: 0.7694500212044648 Specificity: 0.9655492781014003


In [12]:
# Generate the classification report for aggregated results
aggregated_report = classification_report(y_true_agg, y_pred_agg)
print(aggregated_report)

              precision    recall  f1-score   support

           0       0.98      0.97      0.97  42030649
           1       0.61      0.77      0.68   2969351

    accuracy                           0.95  45000000
   macro avg       0.80      0.87      0.83  45000000
weighted avg       0.96      0.95      0.95  45000000



In [15]:
dice = f1_score(y_true_agg, y_pred_agg)
print("Dice Coefficient:", dice)

Dice Coefficient: 0.6803926004902283


In [16]:
auc = roc_auc_score(y_true_agg, y_pred_agg)
print("AUC:", auc)

AUC: 0.8665690728213665


In [17]:
mcc = matthews_corrcoef(y_true_agg, y_pred_agg)
print("MCC:", mcc)

MCC: 0.660031094328147


In [20]:
jaccard = jaccard_score(y_true_agg, y_pred_agg)
print("Jaccard:", jaccard)

Jaccard: 0.5156022925780737


### Parameters tuning

In [33]:
t_values = range(7, 12)  # range for t
A_values = range(100, 301, 50)  # range for A
L_values = range(30, 71, 10)  # range for L

In [39]:
best_params = {}

for t in t_values:
    for A in A_values:
        for L in L_values:

            y_true_agg = []
            y_pred_agg = []

            for image_path in ORIGINAL_IMGs[:1]:
                ID = extract_img_ID(image_path)
                #print(ID)
                
                # Load original image
                img = cv2.imread(image_path)
                
                # Load segmented image, resize to match with segmentation output, select only green channel and flatten 
                img_segment = cv2.imread(SEGMENT_IMG_FOLDER+ID+".tif")
                img_segment_1000 = cv2.resize(img_segment, (1000, 1000))
                y_true = img_segment_1000[:,:,1].flatten() // 255
            
                # Make prediction
                y_pred = retina_segment.segmentation(img, resize=True)
                y_pred = y_pred.flatten() // 255

                # Append the flattened arrays to the aggregated lists
                y_true_agg.extend(y_true)
                y_pred_agg.extend(y_pred)

            # Generate the classification report for aggregated results
            aggregated_report = classification_report(y_true_agg, y_pred_agg)
            auc = roc_auc_score(y_true_agg, y_pred_agg)

            best_params[(t,A,L,auc)] = aggregated_report

In [40]:
for key in best_params.keys():
    print(key)
    print(best_params[key])

(7, 100, 30, 0.8519030827411915)
              precision    recall  f1-score   support

           0       0.97      0.98      0.98    910257
           1       0.76      0.73      0.74     89743

    accuracy                           0.95   1000000
   macro avg       0.87      0.85      0.86   1000000
weighted avg       0.95      0.95      0.95   1000000

(7, 100, 40, 0.8519030827411915)
              precision    recall  f1-score   support

           0       0.97      0.98      0.98    910257
           1       0.76      0.73      0.74     89743

    accuracy                           0.95   1000000
   macro avg       0.87      0.85      0.86   1000000
weighted avg       0.95      0.95      0.95   1000000

(7, 100, 50, 0.8519030827411915)
              precision    recall  f1-score   support

           0       0.97      0.98      0.98    910257
           1       0.76      0.73      0.74     89743

    accuracy                           0.95   1000000
   macro avg       0.87      